In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from dcf import ecf, ytm, iac, bpv, delta, CashFlowList, PayOffModel
from dcf.payoffs import CashFlowDetails, RateCashFlowPayOff

# README issue

In [36]:
from businessdate import BusinessDate, BusinessSchedule
from dcf import pv, CashFlowList
from dcf.plans import amortize, outstanding
from yieldcurves import YieldCurve, DateCurve
BusinessDate.BASE_DATE = 20241001

In [124]:
today = BusinessDate(20201031)
schedule = BusinessSchedule(today, today + "8q", step="1q")
start_date, payment_dates = schedule[0], schedule[1:]

number_of_payments = 8
interest_rate = 0.03
notional = 1000.

plan = amortize(number_of_payments, amount=notional)
out = outstanding(plan, amount=notional)

principal = CashFlowList.from_fixed_cashflows([start_date], [notional])
redemption = CashFlowList.from_fixed_cashflows(payment_dates, plan)
interest = CashFlowList.from_rate_cashflows(payment_dates, out, fixed_rate=interest_rate)
loan = interest - principal + redemption 

In [134]:
BusinessDate.BASE_DATE = 20241015
yield_curve = YieldCurve.from_interpolation([0.0, 5.0, 10.0], [0.03, 0.02, 0.025])
discount_curve = DateCurve(yield_curve, origin=today).df

v = pv(cashflow_list=loan, discount_curve=discount_curve, valuation_date=today)
v, v == 5.77785657421137, today

(2.9759190185570645, False, BusinessDate(20201031))

# `float`

In [4]:
n = 1_000_000
coupon_leg = CashFlowList.from_rate_cashflows([1.,2.,3.,4.,5.], amount_list=n, origin=0., fixed_rate=0.05)
#coupon_leg.print()
redemption_leg = CashFlowList.from_fixed_cashflows([5.], amount_list=n)
# redemption_leg.print()
bond = coupon_leg + redemption_leg
display(bond.table)
print(bond)
print(repr(bond))
bond

[['pay date',
  'cashflow',
  'notional',
  'pay rec',
  'fixed rate',
  'start date',
  'end date',
  'year fraction'],
 [1.0, 50000.0, 1000000, 'pay', 0.05, 0.0, 1.0, 1.0],
 [2.0, 50000.0, 1000000, 'pay', 0.05, 1.0, 2.0, 1.0],
 [3.0, 50000.0, 1000000, 'pay', 0.05, 2.0, 3.0, 1.0],
 [4.0, 50000.0, 1000000, 'pay', 0.05, 3.0, 4.0, 1.0],
 [5.0, 50000.0, 1000000, 'pay', 0.05, 4.0, 5.0, 1.0],
 [5.0, 1000000, None, None, None, None, None, None]]

  pay date     cashflow    notional  pay rec      fixed rate    start date    end date    year fraction
----------  -----------  ----------  ---------  ------------  ------------  ----------  ---------------
       1.0     50_000.0   1_000_000  pay                0.05           0.0         1.0              1.0
       2.0     50_000.0   1_000_000  pay                0.05           1.0         2.0              1.0
       3.0     50_000.0   1_000_000  pay                0.05           2.0         3.0              1.0
       4.0     50_000.0   1_000_000  pay                0.05           3.0         4.0              1.0
       5.0     50_000.0   1_000_000  pay                0.05           4.0         5.0              1.0
       5.0  1_000_000.0
CashFlowList(
[ RateCashFlowPayOff(1.0, 0.0, 1.0, 1000000, fixed_rate=0.05),
  RateCashFlowPayOff(2.0, 1.0, 2.0, 1000000, fixed_rate=0.05),
  RateCashFlowPayOff(3.0, 2.0, 3.0, 1000000, fixed_rate=0.05),
  RateCashFlowPayOff(4.0, 3.0, 4.0, 1000000, 

pay date,cashflow,notional,pay rec,fixed rate,start date,end date,year fraction
1,50000,1000000,pay,0.05,0,1,1
2,50000,1000000,pay,0.05,1,2,1
3,50000,1000000,pay,0.05,2,3,1
4,50000,1000000,pay,0.05,3,4,1
5,50000,1000000,pay,0.05,4,5,1
5,1e+06,,,,,,


### `fit()` with year fraction

In [5]:
from dcf import CashFlowList, fit, pv
from yieldcurves import YieldCurve, DateCurve

In [6]:
today = 0.0
cashflow_list = []
schedule = [1., 2., 3., 4., 5. ]
for i, d in enumerate(schedule):
    pay_dates = [s for s in schedule if s <= d]
    # cf = CashFlowList.from_rate_cashflows(pay_dates, 1_000_000, origin=today, fixed_rate=0.002 * i)
    cf = CashFlowList.from_fixed_cashflows([d], 1)
    cashflow_list.append(cf)

In [7]:
curve = YieldCurve.from_interpolation(schedule, [0.01, 0.009, 0.012, 0.014, 0.011])
targets = [pv(c, curve.df, today) for c in cashflow_list]

In [13]:
fit(cashflow_list, YieldCurve(0.0).df, today, price_list=targets)

{1.0: 0.009999999999989299,
 2.0: 0.008999999998170333,
 3.0: 0.011999999985101466,
 4.0: 0.013999999934628832,
 5.0: 0.011000000000002525}

In [ ]:
yc = YieldCurve(0.0)
fit(cashflow_list, yc.df, today, price_list=targets)

In [ ]:
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve)

In [ ]:
yc = DateCurve(YieldCurve(0.0), origin=0.0)
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve.curve)

In [ ]:
yc = DateCurve(YieldCurve(0.0), origin=0.0)
grid = [yc.year_fraction(max(cf.domain)) for cf in cashflow_list]
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve.curve, fitting_grid=grid)

### `fit()` with BusinessDate

In [ ]:
from businessdate import BusinessDate, BusinessSchedule
from dcf import CashFlowList, fit, pv
from yieldcurves import YieldCurve, DateCurve
from yieldcurves.interpolation import piecewise_linear

In [ ]:
today = BusinessDate(20240101)
cashflow_list = []
schedule = BusinessSchedule(today + '1y', today + '5y', step='1y')
for i, d in enumerate(schedule):
    pay_dates = [s for s in schedule if s <= d]
    #cf = CashFlowList.from_rate_cashflows(pay_dates, 1_000_000, origin=today, fixed_rate=0.002 * i)
    cf = CashFlowList.from_fixed_cashflows([d], 1)
    cashflow_list.append(cf)

In [ ]:
curve = DateCurve(YieldCurve.from_interpolation(schedule, [0.01, 0.009, 0.012, 0.014, 0.011]), origin=today)
targets = [pv(c, curve.df, today) for c in cashflow_list]

In [ ]:
yc = DateCurve(YieldCurve(0.0), origin=today)
fit(cashflow_list, yc.df, today, price_list=targets)

In [ ]:
yc = DateCurve(YieldCurve(0.0), origin=today)
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve.curve)

In [ ]:
yc = DateCurve(YieldCurve(0.0), origin=today)
grid = [yc.year_fraction(max(cf.domain)) for cf in cashflow_list]
fit(cashflow_list, yc.df, today, price_list=targets, fitting_curve=yc.curve.curve, fitting_grid=grid)

### `pv()`

In [ ]:
from yieldcurves import YieldCurve
from curves import plot, lin

In [ ]:
curve = YieldCurve.from_interpolation([0.0], [0.05])
plot(lin(0, 30, 0.2), curve)

In [ ]:
pv(bond, curve.df, 0.0)

### `bpv()`

In [ ]:
from yieldcurves import YieldCurve


In [ ]:
yc = YieldCurve(curve)
pv(bond, yc.df, 0.0)

In [ ]:
pv1 = pv(bond, YieldCurve(curve).df, 0.0)
pv2 = pv(bond, YieldCurve(curve + 0.0001).df, 0.0)
pv2 - pv1

In [ ]:
yc = YieldCurve(curve)

In [ ]:
pv(bond, yc.df, 0.0), bpv(bond, yc.df, 0.0, delta_curve=yc.curve)

### `delta()`

In [ ]:
yc = YieldCurve(curve)

In [ ]:
delta(bond, yc.df, 0.0, delta_curve=yc.curve, delta_grid=[0., 1., 2., 3., 4., 5.])

### `iac()`

In [ ]:
iac(bond, 3.25)

In [ ]:
cf_list = CashFlowList.from_fixed_cashflows([0., 1., 2., 3.], [100, 100, 100, 100])
cf_list *= 10.
cf_list.print() 
#print(cf_list)
#cf_list

In [ ]:
sod = pv(cf_list, curve, valuation_date=0.0)
sod

In [ ]:
sod - ecf(cf_list, 0.0)[0.0]

In [ ]:
float(cf_list()[0.0])

In [ ]:
cl = CashFlowList.from_rate_cashflows([1,2,3,4], 100., origin=0, fixed_rate=0.5)
cl

In [ ]:
cf = RateCashFlowPayOff(pay_date=1.0, start=1.25, end=1.5, amount=100.0, fixed_rate=0.005)
f = lambda *_: 0.05
j = cf.__json__()
print(j)
RateCashFlowPayOff.from_json(j).__copy__()()

In [ ]:
d = dict.fromkeys(map(str, range(6)), 'öakrhgjfhgqjgqklejgrh')
cd = CashFlowDetails(cashflow=1., **d)
#cd.update(d)
cd

### delta()

In [ ]:
from yieldcurves import plotter, YieldCurve
from my.pl import piecewise_linear
c = piecewise_linear([1.,2.,3.], [2.,3.,1.])
plotter[-1:5:0.1](c)

In [ ]:
from dcf import delta
yc = YieldCurve(AlgebraCurve(0.01, inplace=True))
bond_delta = delta(bond, yc.df, 0.0, delta_curve=yc.curve, delta_grid=[0., 2., 4., 6.])
bond_bpv = bpv(bond, yc.df, 0.0, delta_curve=yc.curve)
print(yc)
bond_bpv, sum(bond_delta)

In [ ]:
curve = piecewise_linear([1.,2.,3.], [.02,.03,.01])
yc = YieldCurve(curve)
shift = piecewise_linear([0.0, 1.0, 2.0], [0.0, 0.01, 0.0])
plotter[0:5](curve=curve, shifted_curve=yc.curve+shift, shift=shift)
yc.curve-shift
yc

## pv() with `BusinessDate`

In [ ]:
from businessdate import BusinessDate, BusinessSchedule
today = BusinessDate(20161231)

In [ ]:
from yieldcurves import YieldCurve, plotter
curve = YieldCurve.from_interpolation([0.0, 1.0, 2.0, 3.0, 4.0], [0.01, 0.009, 0.012, 0.014, 0.011])
f = YieldCurve.from_interpolation([0.0], [0.05], spot_price=100.0)  # spot price 100 and yield of 5%
v = YieldCurve.from_interpolation([0.0], [0.1])  # flat volatility of 10%
print(curve(-1.), curve(4.1))
plotter[-1:5](**{r"$\alpha$":curve})

In [ ]:
from yieldcurves import DateCurve
crv = DateCurve(curve, origin=today)
fwd = DateCurve(f, origin=today)
vol = DateCurve(v, origin=today)
vol

In [ ]:
from dcf import OptionPayOffModel
m = OptionPayOffModel.black76(forward_curve=fwd.price, volatility_curve=vol, valuation_date=today)

In [ ]:
from dcf import ecf, pv, CashFlowList, OptionCashFlowPayOff
expiry = today + '3m'
option = OptionCashFlowPayOff(expiry, expiry, amount=-n, strike=110.)
option_list = CashFlowList.from_option_cashflows([expiry], strike_list=110., payoff_model=m)
option = option_list[0]
option

In [ ]:
# option = OptionCashFlowPayOff(expiry, expiry, amount=1, strike=110.)
m(option, today)

In [ ]:
# 0.1025...
ecf(option_list, today, m)

In [ ]:
# 0.1022...
pv(option_list, crv.df, today, m)

# `README`

In [ ]:
from dcf.plans import amortize, outstanding

In [ ]:
today = BusinessDate(20201031)
schedule = BusinessSchedule(today, today + "8q", step="1q")
start_date, payment_dates = schedule[0], schedule[1:]

number_of_payments = 8
interest_rate = 0.001
notional = 1000.

plan = amortize(number_of_payments, amount=notional)
out = outstanding(plan, amount=notional)
out

In [ ]:
principal = CashFlowList.from_fixed_cashflows([start_date], [notional])
principal

In [ ]:
redemption = CashFlowList.from_fixed_cashflows(payment_dates, plan)
redemption

In [ ]:
interest = CashFlowList.from_rate_cashflows(payment_dates, out, fixed_rate=0.001)
interest

In [ ]:
import pandas as pd
loan = interest - principal + redemption 
df = pd.DataFrame.from_records(loan(), index='pay date')
df['cashflow']

In [ ]:
df['df'] = [curve.df(t) for t in df.index]
df['pv'] = df['cashflow'] * df['df']
df

In [ ]:
df['pv'].sum()

In [ ]:
curve = YieldCurve.from_interpolation([today, today + '10y'], [-.005, .005])
curve = DateCurve(curve, origin=today)
x = BusinessSchedule(curve.origin, curve.origin +  '10y', '3m')

In [ ]:
plot(x, **{r"$\gamma(t)$":curve})

In [ ]:
pv(cashflow_list=loan, discount_curve=curve.df, valuation_date=today)  # 4.896613015654154

### `fit(option)`

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from businessdate import BusinessDate
today = BusinessDate(20161231)

In [ ]:
from yieldcurves import YieldCurve, DateCurve, AlgebraCurve
curve = YieldCurve.from_interpolation([0.0, 1.0, 2.0, 3.0, 4.0], [0.01, 0.009, 0.012, 0.014, 0.011])
f = YieldCurve.from_interpolation([0.0], [0.05], spot_price=100.0)  # spot price 100 and yield of 5%
v = YieldCurve.from_interpolation([0.0], [0.1])  # flat volatility of 10%
crv = DateCurve(curve, origin=today)
fwd = DateCurve(f, origin=today)
vol = DateCurve(YieldCurve(AlgebraCurve(v, inplace=True)), origin=today)

In [ ]:
from dcf import ecf, pv, fit, CashFlowList, OptionPayOffModel
expiry = today + '3m'
m = OptionPayOffModel.black76(forward_curve=fwd.price, volatility_curve=vol, valuation_date=today)
option_list = CashFlowList.from_option_cashflows([expiry], strike_list=110., payoff_model=m)

In [ ]:
ecf(option_list, today, payoff_model=m)[expiry]

In [ ]:
pv(option_list, crv.df, today, payoff_model=m)

In [ ]:
vol.curve.curve -= 0.2
pv(option_list, crv.df, today, payoff_model=m)

In [ ]:
vol.curve.curve += 0.2
vol

In [ ]:
from my.pl import piecewise_linear
addon = piecewise_linear([expiry], [0.0], origin=vol.origin)
addon[today + '10y'] = 0.001
addon

In [ ]:
vol

In [ ]:


data = fit([option_list], crv.df, today, fitting_curve=vol.curve.curve, fitting_grid=[expiry], present_value_list=[0.25], addon=addon)